In [ ]:
pip install slack_sdk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os

os.environ['SLACK_API_TOKEN'] = 'API_token'

In [ ]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

def get_all_threads(channel_id, token):

    client = WebClient(token=token)

    try:
        # Call the conversations.history method to fetch all messages in the channel
        result = client.conversations_history(channel=channel_id)

        # Filter out thread messages from the result
        threads = [msg for msg in result['messages'] if 'thread_ts' in msg]

        # Extract thread IDs from the messages
        thread_ids = [thread['ts'] for thread in threads]

        return thread_ids

    except SlackApiError as e:
        print(f"Error: {e.response['error']}")

# Example usage:
channel_id = "C06BZ5PM8SF"
slack_token = "API_token"

thread_ids = get_all_threads(channel_id, slack_token)
print("Thread IDs:", thread_ids)


Thread IDs: ['1710086335.848749', '1710086240.466259', '1710086097.297639', '1710085431.229469', '1710085410.896389', '1710084583.139509', '1710084265.418879', '1709888994.347749', '1708844022.981139', '1708445241.550649', '1708293298.773339', '1708293055.349369', '1708292805.484719', '1708292642.441469', '1708292519.078869', '1708292403.841829']


In [ ]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

def get_messages_in_thread(channel_id, thread_id, token):
    client = WebClient(token=token)
    try:
        result = client.conversations_replies(channel=channel_id, ts=thread_id)
        messages = result['messages']
        return messages
    except SlackApiError as e:
        print(f"Error: {e.response['error']}")


channel_id = "C06BZ5PM8SF"
thread_id = "1710086335.848749"
slack_token = "API_token"

thread_messages = get_messages_in_thread(channel_id, thread_id, slack_token)
print("Messages in Thread:", thread_messages)


Messages in Thread: [{'user': 'U06CMUF9U8G', 'type': 'message', 'ts': '1710086335.848749', 'client_msg_id': '30d1d293-7a45-41f9-9d60-dcfcc2e99b65', 'text': 'Hi <@U06BZ5ZJXGA>', 'team': 'T06BSG4QL86', 'thread_ts': '1710086335.848749', 'reply_count': 1, 'reply_users_count': 1, 'latest_reply': '1710086346.869219', 'reply_users': ['U06BZ5ZJXGA'], 'is_locked': False, 'subscribed': False, 'blocks': [{'type': 'rich_text', 'block_id': 'FkhC4', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'Hi '}, {'type': 'user', 'user_id': 'U06BZ5ZJXGA'}]}]}]}, {'user': 'U06BZ5ZJXGA', 'type': 'message', 'ts': '1710086346.869219', 'bot_id': 'B06BJKFR3PZ', 'app_id': 'A06BZ5ZA7JN', 'text': 'Subject: Revolutionize Your Content Strategy with Alchemyst\n\nDear Alex,\n\nI hope this email finds you well. I am Maya, Market Acquisition Specialist at Alchemyst, a cutting-edge content marketing platform that is transforming the way businesses create, manage, and distribute content at sc

In [ ]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

def send_message_to_thread(channel_id, thread_id, message, token):
    client = WebClient(token=token)
    try:
        response = client.chat_postMessage(channel=channel_id, thread_ts=thread_id, text=message)
        return response
    except SlackApiError as e:
        print(f"Error: {e.response['error']}")


channel_id = "C06BZ5PM8SF"
thread_id = "1710086335.848749"
message = "Hello from the alchemyst!"
slack_token = "API_token"

response = send_message_to_thread(channel_id, thread_id, message, slack_token)
print("Message sent:", response['ok'])


Message sent: True


In [ ]:
!pip install slack_bolt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install schemas

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for schemas: filename=schemas-0.7.1-py3-none-any.whl size=2753 sha256=5f86b213cc4f0be6f632a0dd350c46bb30c16607d3170997c3c6396e5718f07b
  Stored in directory: /root/.cache/pip/wheels/e4/09/93/4aea8059277d05a08c126a8134fab84d74c5b789287bae8836
  Created wheel for functions: filename=functions-0.7.0-py3-none-any.whl size=3036 sha256=5ca94ab0e3711db8763bd273f9056814e3d297c95627807669ba633cf09fcc33
  Stored in directory: /root/.cache/pip/wheels/92/90/bc/01eb2f163bcfa49c7ad5072fbe9d243038368b9001ec26a9da
Successfully built schemas functions


In [ ]:
from logging import Logger
from typing import Any, Dict, List
import time
from slack_bolt import Say

from schemas.slack_schemas import SlackMention, SlackMessage

from client.client import request as client_req
from config import BACKEND_BASE_URL


async def thread_handler(
    context_messages: List[Dict[str, Any]],
    say: Say,
    logger: Logger,
    mention: bool = False,
) -> None:
    """Handle a message in a thread"""
    print("thread_handler")
    print(context_messages)
    # print(body.event.blocks)
    # logger.info(body)
    # await say(
    #     "What's up?"
    # )  # Use say.__call__ to avoid the type checker complaining about the missing thread_ts argument
    message: str = ""

    print("Context messages = ", context_messages)
    if len(context_messages) == 1:
        # message = (
        #     f"Detected a single message in a thread: {context_messages[0]['text']}"
        # )
        message = await client_req(
            method="POST", endpoint="/maya/reply", data={"context": context_messages}
        )
    elif len(context_messages) > 1:
        # message = f"Detected {len(context_messages)} messages in the thread: {list(map(lambda x: x['text'], context_messages))}"
        message = await client_req(
            method="POST", endpoint="/maya/reply", data={"context": context_messages}
        )

    message = message.get("message", "Sorry, I didn't get that.")

    if mention or len(context_messages) > 1:
        # TODO: Write a function to send request to an endpoint that indexes messages in the core platform
        ...

    # NOTE: Log the messages to the logging endpoint in the core.
    await client_req(
        endpoint="/logs/slack",
        method="POST",
        data=[
            {
                "assistant": "Maya",
                "timestamp": time.time(),
                "operations_data": {
                    "context": context_messages,
                    "bot_reply": message,
                    "timestamp": context_messages[0]["ts"],
                    "workspace_id": context_messages[0]["team_id"],
                    "channel": context_messages[0]["event"]["channel"],
                    "is_bot_mentioned": mention,
                },
            },
        ],
    )

    await say(
        message,
        # NOTE: If there is a message in a thread, we need to reply to the thread, else we create a thread by replying to the channel message
        thread_ts=context_messages[0].get("thread_ts", context_messages[0].get("ts")),
    )

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (__init__.py, line 79)